In [1]:
import requests
import os
import bs4
import re
import datetime
import csv

I am setting up this script to only pull filings posted since last run 

In [2]:
last_time_run = datetime.datetime.strptime('01/01/2018', "%m/%d/%Y").date()

In [3]:
form = 'DEF 14A'
base_url = "https://www.sec.gov/cgi-bin/srch-edgar"

In [4]:
url = "%s?text=%s&first=2018&last=2018" %(base_url, form)
list_of_forms = requests.get(url)
soup = bs4.BeautifulSoup(list_of_forms.text, 'html.parser')

In [49]:
table_of_filings = (soup.findAll("table")[4]).findAll("tr")
num_filings = len(table_of_filings)

Loop over all filings on first page

In [86]:
for i in range(1, num_filings):
    row = table_of_filings[i]
    cols = row.findAll("td")
    # Get Company Name and Filing Date
    comp_name = cols[1].get_text()
    filing_date = datetime.datetime.strptime(cols[4].get_text(), "%m/%d/%Y").date()
    # Only grab those filings that have been posted since we last pulled
    if filing_date >= last_time_run:
        txt_url = cols[2].find()['href']
        comp_def14a_html = requests.get('https://www.sec.gov/%s' %txt_url).text
        #form_soup = bs4.BeautifulSoup(comp_def14a_html, 'html.parser')
        #comp_def14a = form_soup.get_text()
        comp_def14a = re.sub('<[^<]+?>', '', comp_def14a_html)
        # If filing doesn't contain median we are not going to find the salary
        if comp_def14a.find("median") == -1:
            pass
            # Write to out file
            #with open('comps_no_median_salary.csv','a', newline='') as nosal_file:
            #    no_sal_writer = csv.writer(nosal_file, delimiter=',')
            #    no_sal_writer.writerow([comp_name, filing_date.year])
            # Typically reported in section called:
                # CEO Pay Ratio or Pay Ratio Disclosure
        else:
            pay_ratio = comp_def14a.rfind("Pay Ratio")
            pay_ratio_caps = comp_def14a.rfind("PAY RATIO")
            # Fairly confident these won't have info, but come back to it
            if pay_ratio == -1 and pay_ratio_caps == -1:
                pass
            # Need to get the section with pay ratio and find median salary
            # May need to build classifier but pretty sure I can retrieve it
            else:
                ratio_sec = max(pay_ratio, pay_ratio_caps)
                pay_section = comp_def14a[ratio_sec : ratio_sec + 5000]
                pay_section = pay_section.strip("\n")
                symbs = ["&", "#", "&nbsp;", "nbsp", "\n", ';']
                for s in symbs:
                    pay_section = pay_section.replace(s, "")
                pay_section = pay_section[0: 5000]
                dollar_signs = [word for word in pay_section.split() if "$" in word]
                print(comp_name)
                print(dollar_signs)
                print(re.search( "[\$]{1}[\d,]+\.?\d{0,2}", 
                                pay_section).groups())

                if len(dollar_signs) == 0:
                    print(pay_section)
                
        

1ST CONSTITUTION BANCORP
['$53,278.160', '$1,487,675,', '$1,000,000', '$1']
()
1ST SOURCE CORP
['(A)$1,855,632Median', 'Murphy)(2)160(B)$45,286Ratio', 'Egidi$62,500160$62,500160$', '82128212$']
()
22nd Century Group, Inc.
['$711,375', '$36,608.', '($/Sh)', 'Awards($)(2)', '$1.39', '$345,987', '$2.12', '$1,903,500', '$1.39', '$128,836', '$1.39', '$128,836', '$1.39', '$64,418(1)Options', '($)', '($)', '$0.69', '$-', '$-', '$0.96']
()
2U, Inc.
['$60,703', 'was$3,389,499']
()
3D SYSTEMS CORP
['$1,933,108', '$55,683.']
()
3M CO
['$20,494,285', '$63,338']
()
A. H. Belo Corp
['compensation$54,142', 'compensation$2,053,650', '$117,000.', '$10,000', '$20,000', 'Cash($)StockAwards($)', 'AllOtherCompensation($)', 'Total($)(a)(b)nb']
()
AAON INC
['$41,582', '$2,027,295.Based', '$41,582.', '($)Chair', '($)Audit', '($)Compensation', '($)Governance', '($)Total', '($)160160160160160160160Jack', 'inCash($)160Restricted160Stock', 'Awards(1)($)160Stock', 'Options($)160All', 'OtherComp.($)160Total($)16016

AttributeError: 'NoneType' object has no attribute 'groups'

In [62]:
has_median = comp_def14a.text.find("median")
print(has_median)

-1


In [ ]:
if has_median == -1:
    comps_wo_med.append(comp_name)
else:
    # Look for Pay Ratio section
    

In [70]:
test_that_has = requests.get('https://www.sec.gov/Archives/edgar/data/1551152/000104746918001843/0001047469-18-001843.txt')

In [74]:
section_start = test_that_has.text.find("Required Pay Ratio")

In [75]:
ratio_paragraph = test_that_has.text[section_start:section_start + 1000]

In [76]:
print(ratio_paragraph)

Required Pay Ratio Disclosure    <BR>    </I></B></FONT></P>

<P style="font-family:arial;"><FONT SIZE=2>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;As required by Section&nbsp;953(b) of the Dodd-Frank Wall Street Reform and Consumer Protection Act and Item&nbsp;402(u) of
Regulation&nbsp;S-K, we are providing the following information about the relationship of the annual total compensation of our employees and the annual total compensation of our CEO, Richard
Gonzalez. The pay ratio included in this information is a reasonable estimate calculated in a manner consistent with Regulation&nbsp;S-K Item&nbsp;402(u). The ratio of Mr.&nbsp;Gonzalez's annual
total compensation for 2017, as reported in the Summary Compensation Table in this proxy statement, to the median employee annual total compensation determined on the same basis was 144:1. For 2017,
the annual total compensation of our median employee (other than Mr.&nbsp;Gonzalez) was $157,347. To i

In [91]:
par_lst = ratio_paragraph.split('. ')

In [92]:
print(par_lst)

['Required Pay Ratio Disclosure    <BR>    </I></B></FONT></P>\n\n<P style="font-family:arial;"><FONT SIZE=2>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;As required by Section&nbsp;953(b) of the Dodd-Frank Wall Street Reform and Consumer Protection Act and Item&nbsp;402(u) of\nRegulation&nbsp;S-K, we are providing the following information about the relationship of the annual total compensation of our employees and the annual total compensation of our CEO, Richard\nGonzalez', 'The pay ratio included in this information is a reasonable estimate calculated in a manner consistent with Regulation&nbsp;S-K Item&nbsp;402(u)', "The ratio of Mr.&nbsp;Gonzalez's annual\ntotal compensation for 2017, as reported in the Summary Compensation Table in this proxy statement, to the median employee annual total compensation determined on the same basis was 144:1", 'For 2017,\nthe annual total compensation of our median employee (other than Mr.&nbsp;Gonzalez) was 

In [116]:
for sentence in par_lst:
    if sentence.find("median employee") != -1 and sentence.find("$") != -1:
        #med_salary = re.search('\$[0-9]+', sentence)
        med_salary = re.search('\$\d+,\d+\d+?', sentence)
        print(med_salary.group())

$157,347


In [120]:
comps_w_med.loc[0] = ['AbbVie', med_salary.group()]


In [121]:
comps_w_med.head()

,company,med_emp_salary
0,AbbVie,"$157,347"
